## Download an image from the archive, display it and find sources in it.

### Outline

- Download an image from the Hubble archive (either from the web interface or with astroquery)
- Learn about FITS format
- Display the image in ds9 (play with ds9 features)
- Display in matplotlib
- Use photutils.DAOStarFinder to find objects in the image
- Save the result to a file.

### Barbara A. Mikulski Archive For Space Telescopes (MAST)

https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html

Download a 47 Tuc image with WFC3/UVIS.

$ ls HST

ib2j02020_asn.fits	ib2j02020_drc.fits	ib2j02020_drz.fits

#### FITS Format

Flexible Image Transport System (FITS) is an open standard defining a digital file format useful for storage, transmission and processing of data.

Image metadata is stored in a human-readable ASCII header. "metadata" reffers to data that describes other data. The header describes the data array that follows. A FITS file may contain multiple Header-Data Units (HDUs).

The library to read/write FITS is in the **astropy** package.

In [ ]:
from astropy.io import fits

Let's open the "drz" file and look at  its contents 

In [ ]:
f = fits.open('ib2j02020_drz.fits')
f

The `info` method gives an overview of the ocntents of a FITS file:

In [ ]:
f.info()

The empty parenthesis "()" in the "Dimensions" column indicates that the 0-th HDU has no data. This is called the "Primary Header" and ususally information common  to all extensions in a file is stored there. FITS headers store "cards". A card consists of a 8 char keyword, a value and a comment. 

In [ ]:
f[0].header

The first extension (or HDU) is called `SCI` and it stores the science data and metadata.

In [ ]:
f[1].header

FITS headers are like Python dictionaries. We can get the value of a keyword by using th ekeyword as a dict key:

In [ ]:
f[1].header['RA*']

**Exercise:**

Using wild cards find all keywords starting with `C` in the header of the first HDU.

To access the data we use the index of the HDU we want to access, followed by the `data` attribute.

In [ ]:
f[1].data

Let's look at the data. There are several ways to display astronomical images. 
The most popular one is a command line program called `ds9`.
We will look at it later.

Now we will use a python package, perhaps the most popular, for plotting, called `matplotlib`.

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'viridis'

# Run the %matplotlib magic command to enable inline plotting
# in the current notebook.  Choose one of these:
%matplotlib inline
%matplotlib notebook

plt.imshow(f[1].data)

The reason we don't see anything is that that the range of values in the image is too big.
From the `matplotlib` documentation we see there are two optional parameters:

```
vmin, vmax : scalar, optional
When using scalar data and no explicit *norm*, *vmin* and *vmax*
define the data range that the colormap covers. By default,
the colormap covers the complete value range of the supplied
data. *vmin*, *vmax* are ignored if the *norm* parameter is used.
```

In order to "see" the image we need to define some reasonable values for the range of data.
We can use the `stats` subpackage in `astropy`.

In [ ]:
from astropy.stats import sigma_clipped_stats

data = f[1].data
mean, median, std = sigma_clipped_stats(data, sigma=3.0)  
print((mean, median, std))  

In [ ]:
#sigma_clipped_stats?


In [ ]:
# Other ways to get statistics of a data array

import numpy as np

data.mean()
np.nanmean(data)

In [ ]:
plt.imshow(f[1].data, vmin=0.1, vmax=.4, aspect="auto")

### Find sources in the image

We will use the `photutils` package to find the location and flux of sources in the image.

In [ ]:
from photutils import DAOStarFinder

`fwmh` - full width at half maximum

In [ ]:
from astropy.modeling.models import Gaussian1D
import numpy as np
gauss = Gaussian1D(10, 4, .5)
x = np.arange(10)
y = gauss(x)
#%matplotlib inline
#plt.plot(x, y)


In [ ]:
daofind = DAOStarFinder(fwhm=3.0, threshold=5.*std)  
sources = daofind(data - median)
type(sources)

In [ ]:
for col in sources.colnames:  
    sources[col].info.format = '%.8g'  # for consistent table output
print(sources)  

The result, `sources`, is an astropy table. Let's look at the first row and discuss the columns and data types.

In [ ]:
sources[0]

### How to get information about a function or an object?

In [ ]:
DAOStarFinder?


In [ ]:
type(sources)

In [ ]:
sources.write('sources.csv')